<a href="https://colab.research.google.com/github/KJTDHISAW/Fine_Tuning_PlinAI/blob/main/Mahna_Mahna%2C_Fine_Tuning_Mistral_7b_in_Google_Colab_with_QLoRA(fine_tuning_attempt_0.1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Mahna_Mahna')

Mahna_Mahna


# KT First Finetuning Attempt (first_adjustment_0.1)

based off Ali Mobarekati, 2024 ["Fine-Tuning Mistral 7b in Google Colab with QLoRA (complete guide)"](https://medium.com/@codersama/fine-tuning-mistral-7b-in-google-colab-with-qlora-complete-guide-60e12d437cca).


##Imports etc

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
#set huggingface_token
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

In [ ]:
#model
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "Enlighten_Instruct"

test_path='/content/Enlighten-Instruct/Dataset/TestData.csv'
train_path='/content/Enlighten-Instruct/Dataset/TrainData.csv'

In [ ]:
#necessary? Mobarekati uses the cell above, while Heath below. Will try both,
model = "mistralai/Mixtral-8x7B-Instruct-v0.1" # see https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

client = InferenceClient(model) # see https://huggingface.co/docs/huggingface_hub/package_reference/inference_client


In [ ]:
#downloads
%%capture
!git clone 'https://github.com/ali7919/Enlighten-Instruct.git'
!pip install -U bitsandbytes
!pip install --upgrade transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

In [ ]:
#loads
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re
import torch
from torch.utils.data import Dataset


In [ ]:
#mount googledrive
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive

In [ ]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Load training data

In [ ]:
csv_path = '/content/drive/MyDrive/training_data7_SH_prompt.csv'
df = pd.read_csv(csv_path)

print(df)

                                             instruction  \
0      Identify and then include in your response a p...   
1      Identify and then include in your response a p...   
2      Identify and then include in your response a p...   
3      Identify and then include in your response a p...   
4      Identify and then include in your response a p...   
...                                                  ...   
57135  Identify and then include in your response a p...   
57136  Identify and then include in your response a p...   
57137  Identify and then include in your response a p...   
57138  Identify and then include in your response a p...   
57139  Identify and then include in your response a p...   

                                         expected_output  \
0      At plin. nat. 1.1, pliny_the_elder wrote the f...   
1      At plin. nat. 1.1, pliny_the_elder wrote the f...   
2      At plin. nat. 1.1, pliny_the_elder wrote the f...   
3      At plin. nat. 1.1, pliny_the_eld

In [ ]:
#dataset to trainable format
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.data = self._process_dataframe()

    def _process_dataframe(self):
        data_for_training = []

        for index, row in self.dataframe.iterrows():
            text = '<s>[INST]@Enlighten. ' + row['instruction'] + '[/INST]' + row['expected_output'] + '</s>'
            conversation_input = [
                self.tokenizer.bos_token_id,
                *self.tokenizer.encode("[INST]@Enlighten. " + row['instruction'] + "[/INST]" + row['expected_output']),
                self.tokenizer.eos_token_id,
            ]

            data_point = {
                'text': text,
                'conversation_input': conversation_input,
                'input_ids': conversation_input,
            }

            data_for_training.append(data_point)

        return data_for_training

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

custom_dataset = CustomDataset(dataframe=df, tokenizer=tokenizer)

In [ ]:
# Print the first few items in custom_dataset
for i in range(min(5, len(custom_dataset))):
    print(custom_dataset[i])


{'text': '<s>[INST]@Enlighten. Identify and then include in your response a passage of Pliny that addresses the following topic: terra according to CountVectorizer[/INST]At plin. nat. 1.1, pliny_the_elder wrote the following: libro ii continentvr an finitus sit mundus et an unus de forma eius de motu eius cur mundus dicatur de elementis de deo de siderum errantium natura de lunae et solis defectibus de nocte de magnitudine siderum quae quis invenerit in observatione caelesti de lunae motu errantium motus et luminum canonica quare eadem altiora alias propiora videantur catholica siderum errantium quae ratio colores eorum mutet solis motus dierum inaequalitatis ratio quare fulmina iovi adsignentur intervalla siderum de sideribus musica de mundo geometrica de repentinis sideribus de cometis natura et situs et genera eorum hipparchea de sideribus agnoscendis  de caelestibus prodigiis per exempla historica lampades bolides trabes caelestes chasma caeli de caeli coloribus de flamma caelesti 

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)


model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

('<s>', '</s>')

## adapters, hyperparameter, parameters

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=custom_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
from torch.utils.checkpoint import checkpoint_sequential
from your_module import BitsAndBytesConfig, LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Assuming you have defined your custom model, you can create a checkpointed version
checkpointed_model = lambda x: checkpoint_sequential(model(x), segments, input_indices, output_indices)

# Update the trainer to use the checkpointed model
trainer.model = checkpointed_model

# Training loop
trainer.train()


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.70 GiB. GPU 0 has a total capacty of 14.75 GiB of which 7.35 GiB is free. Process 6213 has 7.39 GiB memory in use. Of the allocated memory 7.26 GiB is allocated by PyTorch, and 8.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [ ]:
trainer.model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KJTDHISAW/Enlighten_Instruct/commit/0a7a6fad2e7de15df80ec3b0afb1bc494ed86d76', commit_message='Upload model', commit_description='', oid='0a7a6fad2e7de15df80ec3b0afb1bc494ed86d76', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drive_path = '/content/drive/MyDrive/'
trainer.model.save_pretrained(drive_path)


Test the Model

In [ ]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST]@Enlighten. {question} [/INST]"
  return prompt

In [ ]:
question = "what is the pwoer outage event?"
prompt = build_prompt(question)
result = pipe(prompt)

print(result[0]['generated_text'])